In [1]:
from wovenv.ai.q_learning import QLearningAgent
from wovenv.venv.env import Env
from IPython.display import clear_output

# Окружение и агент

In [2]:
env = Env()
agent = QLearningAgent(alpha=1, discount=0.5)

In [3]:
def generate_session(env: Env, agent: QLearningAgent, t_max=1000, train=False):
    env.write_log = not train
    total_reward = 0
    s = env.reset()

    game = []

    for t in range(t_max):
        a = agent.get_action(s)
        next_s, r, done = env.step(a)

        game.append((s, a, next_s, r, done))

        total_reward += r
        s = next_s

        if done:
            if not train: env.finish_log()
            break

    if train:
        for s, a, next_s, r, done in game:
            agent.update(s, a, next_s, r, done)

    return total_reward

# Горячий старт (при необходимости)

In [4]:
def hot_start(env: Env, agent: QLearningAgent, ind, t_max=1000):
    clear_output()
    print(f'start {ind}')
    s = env.reset()

    for t in range(t_max):
        acs = env._get_actions()
        for a in acs:
            next_s, r, done = env.step(a)
            agent.update(s, a, next_s, r, done)
            s = next_s
            if done: return

In [5]:
#for i in range(1000):
#    hot_start(env, agent, i)

# Обучение

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
from collections import deque

for _ in range(100):
    agent.epsilon = 1

    rewards = deque()
    while agent.epsilon > 0.1:
        rewards.append(generate_session(env,agent,train=True))
        agent.epsilon *= 0.99
        clear_output(True)
        print(agent.epsilon)
        plt.plot(rewards)
        plt.show()

# Сериализация

In [ ]:
import pickle

---
### Сохранение обученного агента

In [ ]:
with open('data.pickle', "wb") as f:
    pickle.dump(agent, f)

---
### Загрузка сохраненного агента

In [ ]:
agent = pickle.load(open('data.pickle', 'rb'))

# Генерация игры с записью в лог для последующей визуализации

In [ ]:
agent.epsilon = 0
generate_session(env, agent)